In [2]:
from root_pandas import read_root
import numpy as np
from ROOT import TFile,TTree
import matplotlib.pyplot as plt
from scipy.optimize import curve_fit

/home/jcordero/anaconda3/lib/python3.6/site-packages/root_numpy/__init__.py:46: RuntimeWarning: numpy 1.14.3 is currently installed but you installed root_numpy against numpy 1.12.1. Please consider reinstalling root_numpy for this numpy version.
  RuntimeWarning)


In [3]:
def exp2(x,*param):
    lamb, x0, N ,y0 = param 
    return N*np.exp(-lamb*(x - x0)) + y0 
def exp(x,*param):
    lamb, x0, N = param 
    return N*np.exp(-lamb*(x - x0))
def poly2(x,*param):
    return param[0] + param[1]*x + param[2]*x*x

def chi2(x,sig,func,parm):
    chi = 0
    for xi,si in zip(x,sig):
        chi+= ((func(x,*parm)-xi)**2)/(si)**2
    return chi

In [4]:
Path='/home/jcordero/CMS/BRIL/data/Cluster/'
x,y,z = [],[],[]
phi,theta = [],[]
merge = []

file,tree = {},{}

PU = ['0','1','10','20','75']
#PU = ['20']
#PU=['2']

for pu in PU:
    #files[pu] = read_root(Path+'Cluster_'+pu+'.0.root')
    #files = TFile(Path+'Cluster_'+pu+'.0.root')
    #files = TFile(Path+'Cluster_'+pu+'.root')
    files = TFile(Path+'Cluster_'+pu+'.0_01.root')
    tree[pu] = files.Get('cluster_tree') 
    for t in tree[pu]:
        x.append(t.CluX)
        y.append(t.CluY)
        z.append(t.CluZ)
        phi.append(t.CluPhi)
        theta.append(t.CluTheta)
        merge.append(t.CluMerge)

TClass::Init:0: RuntimeWarning: no dictionary for class ROOT::TIOFeatures is available


KeyboardInterrupt: 

In [4]:
DF = read_root(Path+'Cluster_'+pu+'.root')

OSError: cannot read /home/jcordero/CMS/BRIL/data/Cluster/Cluster_2.root

In [ ]:
x      = np.array(x)
y      = np.array(y)
z      = np.array(z)
phi    = np.array(phi)
theta  = np.array(theta)
merge  = np.array(merge)
r      = np.sqrt(x**2 + y**2)

In [ ]:
'''
rMaskRange = {
                1:[6.5,9.8],
                2:[10.9,13.6],
                3:[14.6,17.3],
                4:[18.3,20.9],
                5:[21.9,25.3],
                }
''';
rMaskRange = {
                1:[6.5,9.7],
                2:[10.9,13.7],
                3:[14.5,17.3],
                4:[18.4,20.9],
                5:[21.9,25.3],
                }

zMaskRange = {
                1:[174,176],
                2:[200,202],
                3:[230,232],
                4:[264,266],
                }

zMaskRangeInner = {
                    1:{
                        1:[174  ,174.5],
                        2:[174.5,175  ],
                        3:[175  ,175.5],
                        4:[175.5,176 ],
                    },
                    2:{
                        1:[200  ,200.5],
                        2:[200.5,201  ],
                        3:[201  ,201.5],
                        4:[202.5,203  ],
                    },
                    3:{
                        1:[230  ,230.5],
                        2:[230.5,230.7],
                        3:[231.7,231.2],
                        4:[231.2,231.6],
                    },
                    4:{
                        1:[264  ,264.5],
                        2:[264.5,265  ],
                        3:[265  ,265.5],
                        4:[265.5,266   ],
                    },
                    }
    
NRing = {
        1:10,
        2:14,
        3:18,
        4:22,
        5:24
        }    

In [ ]:
Disk = {}
InnerDisk = {}
Ring = {}
for zi in zMaskRange:
    PosD= np.logical_and( z > zMaskRange[zi][0] , z < zMaskRange[zi][1])
    NegD= np.logical_and( z < -zMaskRange[zi][0] , z > -zMaskRange[zi][1])
    NegD = PosD
    Disk[zi] = np.logical_or(PosD,NegD) 
    InnerDisk[zi] = {}
    for inzi in zMaskRangeInner[zi]:
        InnerPosD = np.logical_and( z > zMaskRangeInner[zi][inzi][0] , z < zMaskRangeInner[zi][inzi][1])
        InnerNegD = np.logical_and( z < -zMaskRangeInner[zi][inzi][0] , z > -zMaskRangeInner[zi][inzi][1])
        InnerNegD = InnerPosD 
        InnerDisk[zi][inzi] = np.logical_or(InnerPosD, InnerNegD)
        
for ri in rMaskRange:
    Ring[ri] = np.logical_and( r > rMaskRange[ri][0] , r < rMaskRange[ri][1])

In [ ]:
mask = []
n = 0
#P0 = [1,0,2000,0]
P0 = [1,0,1000]

plt.figure(figsize=(10,10))
for di in Disk:
    for ri in Ring:
        #mask[n] = np.logical_and(Disk[di],Ring[ri]) 
        mask = np.logical_and(Disk[di],Ring[ri]) 
        n+=1
        plt.subplot(4,5,n)
        hist = plt.hist(
                        r[mask],
                        bins = 40,
                        histtype = 'step',
                        )
        plt.title('Disk '+str(di)+' Ring '+str(ri))
        plt.grid(linestyle='--')
        
        X,Y = hist[1],hist[0]
        xc = (X[:-1]+X[1:])/2
        
        #func = exp
        func = poly2
        parm = curve_fit(func,xc,Y,p0=P0)
        plt.plot(xc,func(xc,*parm[0]))
        
        sig = []
        for i in range(len(parm[1])):
            sig.append(parm[1][i][i])
        #print(chi2(xc,sig,func,parm))
        
plt.tight_layout()
        

##### hist

In [ ]:
mask = []
n = 0
#P0 = [1,0,2000,0]
P0 = [1,0,1000]

plt.figure(figsize=(10,10))
for di in Disk:
    mask = Disk[di]
    mask_nonOverlap = Ring[1]
    for ri in range(2,len(Ring)):
        mask_nonOverlap = np.logical_or(mask_nonOverlap,Ring[ri])
    mask = np.logical_and(mask,mask_nonOverlap)
    
    n+=1
    plt.subplot(2,2,n)
    hist = plt.hist(
                    r[mask],
                    bins = 40,
                    histtype = 'step',
                    )
    plt.title('Disk '+str(di))
    plt.grid(linestyle='--')

    X,Y = hist[1],hist[0]
    xc = (X[:-1]+X[1:])/2

    #func = exp
    func = poly2
    parm = curve_fit(func,xc,Y,p0=P0)
    plt.plot(xc,func(xc,*parm[0]))

    sig = []
    for i in range(len(parm[1])):
        sig.append(parm[1][i][i])
    #print(chi2(xc,sig,func,parm))
        
plt.tight_layout()

In [ ]:
mask= [True for _ in x]
plt.figure(figsize=(10,10))
h=plt.hist2d(x[mask],y[mask],bins = 1000);
################
x0=0.5
#X = np.arange(x0,2,step=0.1)
Y = np.arange(0,26,step=0.1)
m = 20
Xmin = Y/m+x0
plt.plot(Xmin,Y,'r')
###############
x0=0.8
m = 20
Xmax = Y/m+x0
plt.plot(Xmax,Y,'y')

#maskX = np.logical_and(x>min(Xmin),x < max(Xmax))
#maskY = np.logical_and(y>min(Y),y < max(Y))
#mask = np.logical_and(maskX,maskY)

xc = h[1][:-1] + h[1][1:]
yc = h[2][:-1] + h[2][1:]
mask = np.logical_and(x-y/m > 0.5 , x-y/m < 0.8)
mask = np.logical_and(mask,y>0)

plt.figure(figsize=(10,10))
plt.hist2d(x[mask],y[mask],bins = 1000);
ax = plt.gca()
ax.set_xlim([-26,26])
ax.set_ylim([-26,26])

In [ ]:
################
x0=0.5
Y = np.arange(0,max(y),step=max(y)/1000.0)
m = 20
Xmin = Y/m+x0
###############
x0=0.8
m = 20
Xmax = Y/m+x0
###############

xc = (h[1][:-1] + h[1][1:])/2
yc = (h[2][:-1] + h[2][1:])/2
ind = np.logical_and(x-y/m > 0.5 , x-y/m < 0.8)
plt.hist(r[ind],
         bins = 30,
       )
plt.show();

In [ ]:
d = 2
mask= np.logical_or(InnerDisk[d][1],InnerDisk[d][2])
plt.figure(figsize=(10,10))
plt.hist2d(x[mask],y[mask],bins = 300);

In [ ]:
d = 1
mask = InnerDisk[d][1]
#mask= np.logical_or(InnerDisk[d][3],InnerDisk[d][4])
#mask= np.logical_and(InnerDisk[d][3],InnerDisk[d][4])

######################################################
#intX12 = np.intersect1d(x[InnerDisk[d][1]],x[InnerDisk[d][2]])
#intX23 = np.intersect1d(x[InnerDisk[d][2]],x[InnerDisk[d][3]])
#intX34 = np.intersect1d(x[InnerDisk[d][3]],x[InnerDisk[d][4]])
#xinc = [(xi in intX12) and (xi in x[InnerDisk[d][2]]) for xi in x]
#xinc = [(xi in intX12)  for xi in x]
###########################
#intY12 = np.intersect1d(y[InnerDisk[d][1]],y[InnerDisk[d][2]])
#intY23 = np.intersect1d(y[InnerDisk[d][2]],y[InnerDisk[d][3]])
#intY34 = np.intersect1d(y[InnerDisk[d][3]],y[InnerDisk[d][4]])
#yinc = [(yi in intY12) and (yi in y[InnerDisk[d][2]]) for yi in y]
#yinc = [(yi in intY12)  for yi in y]

#maskInt = np.logical_and(xinc,yinc)
#mask = np.logical_and(xinc,yinc)
######################################################

#mask = np.logical_and(mask,maskInt)
plt.figure(figsize=(10,10))
plt.hist2d(x[mask],y[mask],bins = 300);

################
x0=-0.3
#X = np.arange(x0,2,step=0.1)
Y = np.arange(0,26,step=0.1)
m = 15
Xmin = Y/m+x0
plt.plot(Xmin,Y,'r')
###############
x0=0.3
m = 15
Xmax = Y/m+x0
plt.plot(Xmax,Y,'y')

In [ ]:
d = 1
mask= Disk[d]
plt.figure(figsize=(10,10))
plt.hist2d(x[mask],y[mask],bins = 1000);



In [ ]:
mask = []
n = 0
#P0 = [1,0,2000,0]
P0 = [1,0,1000]

#################
x0=0.6
Y = np.arange(0,26,step=26/len(y))
m = 20
Xmin = Y/m+x0
#################
x0=0.7
m = 20
Xmax = Y/m+x0
##################


plt.figure(figsize=(10,10))
for di in Disk:
    mask = Disk[di]
    #mask = [True for _ in x]
    mask_nonOverlap = Ring[1]
    for ri in range(2,len(Ring)):
        mask_nonOverlap = np.logical_or(mask_nonOverlap,Ring[ri])
    mask = np.logical_and(mask,mask_nonOverlap)
    
    #maskPos = np.logical_and(x-y/m > 0.5, x-y/m <0.8)
    
    #mask = np.logical_and(mask,maskPos)
    
    
    n+=1
    plt.subplot(2,2,n)
    hist = plt.hist(
                    r[mask],
                    bins = 40,
                    histtype = 'step',
                    )
    plt.title('Disk '+str(di))
    plt.grid(linestyle='--')

    X,Y = hist[1],hist[0]
    Xc = (X[:-1]+X[1:])/2
    xc,yc = [],[]
    for i in range(len(Y)):
        if Y[i]!=0:
            xc.append(Xc[i])
            yc.append(Y[i])

    #func = exp
    func = exp2
    #func = poly2
    if func == exp2:
        P0 = [0.2,7,200,0]
    elif func == exp:
        P0 = [0.2,7,200]
    elif func == poly:
        P0 = [10,-1,10]   
    
    plt.plot(xc,yc)
    parm = curve_fit(func,xc,yc,p0=P0)
    plt.plot(xc,func(xc,*parm[0]))

    sig = []
    for i in range(len(parm[1])):
        sig.append(parm[1][i][i])
    #print(chi2(xc,sig,func,parm))
        
plt.tight_layout()

In [ ]:
plt.figure(figsize=(10,10))
plt.hist2d(x[mask],y[mask],bins=1000)

In [ ]:
n = 0
plt.figure(figsize=(10,10))
for di in Disk:
    mask = Disk[di]
    mask_nonOverlap = Ring[1]
    #for ri in range(2,len(Ring)):
    #    mask_nonOverlap = np.logical_or(mask_nonOverlap,Ring[ri])
    mask = np.logical_and(mask,mask_nonOverlap)
    
    n+=1
    plt.subplot(2,2,n)
    plt.hist(
            phi[mask],bins = 100,
            histtype = 'step'
            )

    
plt.figure(figsize=(10,10))    
plt.hist2d(x[mask],y[mask],bins = 1000)
plt.show();

In [ ]:
d,din = 1,2
R = []

plt.figure(figsize=(7,7))
for di in [1,2]:#InnerDisk:
    #plt.figure()
    #Mask = InnerDisk[d][di]
    Mask = np.logical_or(InnerDisk[d][di],InnerDisk[d][di+2])
    mask_nonOverlap = Ring[1]
    for ri in range(2,len(Ring)+1):
        mask_nonOverlap = np.logical_or(mask_nonOverlap,Ring[ri])
    Mask = np.logical_and(Mask,mask_nonOverlap)
    
    for ri in Ring:
        mask = Ring[ri]
        mask = np.logical_and(mask,Mask)
        
        h = plt.hist(r[mask],
                histtype = 'step',
                weights = [1/NRing[ri] for _ in r[mask]],
                range = [5,27],
                bins = (27-5)*2,    
                )
        R.append(h)
plt.grid(linestyle='--')
        
    





In [ ]:
d = 1
R,wei = [],[]

plt.figure(figsize=(7,7))

mask = [False for _ in z]
mask_nonOverlap = [False for _ in z]

for ri in Ring:
    mask_nonOverlap = np.logical_or(mask_nonOverlap,Ring[ri])
    Mask = np.logical_or(mask,Disk[d])
    mask = np.logical_and(Mask,Ring[ri])
    R.append(r[mask])
    wei.append([1/NRing[ri] for _ in r[mask]])
    
h = plt.hist(
        R,
        histtype = 'step',
        weights = wei,
        range = [6,26],
        bins = 200,    
        )    

plt.grid(linestyle='--')
        
    





In [ ]:
data = np.array(sum(h[0]))
mask = data!=0

xc = (h[1][:-1]+h[1][1:])/2
plt.plot(xc[mask],data[mask])
func = exp2
parm = curve_fit(func,xc,data,p0=[2,6,500,0])
plt.plot(xc,func(xc,*parm[0]),'-k')
plt.plot(xc,func(xc,*[]),'-k')

In [ ]:
data